In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [2]:
# load IRIS dataset
dataset = pd.read_csv('https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv')

# transform species to numerics
dataset.loc[dataset.species=='setosa', 'species'] = 0
dataset.loc[dataset.species=='versicolor', 'species'] = 1
dataset.loc[dataset.species=='virginica', 'species'] = 2

X = dataset.drop(columns=['species'])
y = dataset.species.astype('int')

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)

# wrap up with Variable in pytorch
train_X = Variable(torch.Tensor(train_X.values).float())
test_X = Variable(torch.Tensor(test_X.values).float())
train_y = torch.Tensor(train_y.values)
test_y = torch.Tensor(test_y.values)


In [6]:

class Net(nn.Module):
    # define nn
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = self.fc2(X)
        X = self.fc3(X)
        X = self.softmax(X)

        return X
    


net = Net()

criterion = nn.CrossEntropyLoss()# cross entropy loss

optimizer = torch.optim.SGD(net.parameters(), lr=0.05)

for epoch in range(1000):
    optimizer.zero_grad()
    out = net(train_X)
    loss = criterion(out, train_y.long())
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('number of epoch', epoch, 'loss', loss.data.item())

predict_out = net(test_X)
_, predict_y = torch.max(predict_out, 1)

print('prediction accuracy', accuracy_score(test_y.data, predict_y.data))



number of epoch 0 loss 1.1339986324310303
number of epoch 100 loss 0.7269532084465027
number of epoch 200 loss 0.6533516049385071
number of epoch 300 loss 0.6110961437225342
number of epoch 400 loss 0.5997253656387329
number of epoch 500 loss 0.5941113233566284
number of epoch 600 loss 0.5904865264892578
number of epoch 700 loss 0.5879192352294922
number of epoch 800 loss 0.5859654545783997
number of epoch 900 loss 0.5843396782875061
prediction accuracy 1.0


In [7]:
print('macro precision', precision_score(test_y.data, predict_y.data, average='macro'))
print('micro precision', precision_score(test_y.data, predict_y.data, average='micro'))
print('macro recall', recall_score(test_y.data, predict_y.data, average='macro'))
print('micro recall', recall_score(test_y.data, predict_y.data, average='micro'))

macro precision 1.0
micro precision 1.0
macro recall 1.0
micro recall 1.0
